- RDD storage levels
- Caching y persistencia distribuida de RDDs
- RDDs Checkpointing
- Escritura de RDD a archivos

## RDD lineage

![RDD lineage](https://github.com/israelzuniga/dlatam-bigdata-workshop/blob/master/notebooks/img/rdd_lineage.png?raw=true)

In [ ]:
!wget https://raw.githubusercontent.com/israelzuniga/dlatam-bigdata-workshop/master/notebooks/data/lorem.txt

In [ ]:
APP_NAME = "RDDS"
SPARK_URL = "local[*]"


from pyspark import SparkConf, SparkContext

In [ ]:
conf = (SparkConf()\
        .setMaster(SPARK_URL)\
        .setAppName(APP_NAME))



sc = SparkContext(conf= conf)

In [ ]:
# File: lorem.txt
lorem = sc.textFile('lorem.txt')

In [ ]:
#RDD: lorem

words = lorem.flatMap(lambda x: x.split())


words.take(12)


In [ ]:
# RDD: words

longwords = words.filter(lambda x: len(x) > 5)



longwords.take(6)

In [ ]:
# RDD: longwords

numwords = longwords.count()


print(numwords)

In [ ]:
print(longwords.toDebugString())

La acción `longwords.count()` obliga la evaluación de los RDDs padres hasta `longwords`. Si esta acción (o cualquier otra como `longwords.take(6)` o `longwords.collect()`) es llamada en una ocasión posterior, el linaje entero se reevualua . En casos simples, con datos pequeños en una o dos fases, las reevaluaciones no son problema. Pero en muchas circunstancias pueden ser ineficientes y pueden impactar el tiempo de recuperación en caso de catástrofe.

## RDD Storage Levels


Storage Level|Desc
------------|------
MEMORY_ONLY|(Default) RDD partitions are stored in memory only.
MEMORY_AND_DISK| RDD partitions that do no fit in memory are stored in disk.
MEMORY_ONLY_SER| RDD partitions are stored as serialized objects in memory. This option can be used to save memory (as serialized objects may consume less space than their deserialized equvalent).
MEMORY_AND_DISK_SER| RDD partitions are stored as serialized objects in memory. Objects that do not fit into memory are spilled to disk.
DISK_ONLY| RDD partitions are stored on disk only.




## Storage Level Flags

### StorageClass Constructor
```
StorageLevel(useDisk,
              useMemory,
              useOffHeap,
              deserialized,
              replication=1)
```


`useDisk`, `useMemory`, `useOffHeap`, y `deserialized` son argumentos Booleanos, mientras que `replication` es de valor entero (default a 1)


### Spark API: `getStorageLevel()`

In [ ]:
lorem.getStorageLevel()

In [ ]:
lorem_sl = lorem.getStorageLevel()

In [ ]:
lorem_sl.useDisk

In [ ]:
lorem_sl.useMemory

## Eligiendo un nivel de almacenamiento:

El nivel de almacenamiento de los RDD permiten ajustar el funcionamiento de los trabajos en Spark y acomodar operaciones que de otra forma no tendrían espacio en la memoria del cluster. Adicionalmente, las opciones disponibles de replicación pueden reducir el tiempo de restauraación en caso de fallas.

Generalmente hablando, si un RDD cabe en la memoria disponible del cluster, el nivel de almacenamiento por default es suficiente y proveerá el mejor rendimiento.


-----




# Caching / Persistence / Checkpointing


## Caching

In [ ]:
!wget https://raw.githubusercontent.com/israelzuniga/dlatam-bigdata-workshop/master/notebooks/data/all-shakespeare.txt

In [ ]:
doc = sc.textFile("all-shakespeare.txt") 

In [ ]:
words = doc.flatMap( lambda x: x.split()) \
    .map( lambda x: (x, 1))\
    .reduceByKey( lambda x, y: x + y) 


In [ ]:
words.cache() 
words.count() # triggers computation 

In [ ]:
words.take( 30) # no computation required 

In [ ]:
words.count() # no computation required 

## Persistence `persist()`

```python
# Default
RDD.persist(storageLevel =  StorageLevel.MEMORY_ONLY_SER)

# myrdd
myrdd.persist(StorageLevel.MEMORY_AND_DISK_SER_2)

# =
myrdd.persist(StorageLevel(True, True, False, False, 2))
```


##  `unpersist()`
Si el RDD deja de requerir perssitencia en disco, usamos este método. También si queremos cambiar las propiedades de persistencia de un RDD, debemos abandonar su configuración y especificarla posteriormente.

In [ ]:
doc = sc.textFile("all-shakespeare.txt") 

In [ ]:
words = doc.flatMap( lambda x: x.split()) \
    .map( lambda x: (x, 1))\
    .reduceByKey( lambda x, y: x + y) 


In [ ]:
words.persist()

In [ ]:
words.count()

In [ ]:
words.take(3)

In [ ]:
print(words.toDebugString())

In [ ]:
words.getStorageLevel()

In [ ]:
from pyspark import StorageLevel

In [ ]:
words.persist(storageLevel=StorageLevel(False, True, False, False, 10))

In [ ]:
words.unpersist()

In [ ]:
words.persist(storageLevel=StorageLevel(False, True, False, False, 10))

## Checkpointing `checkpoint()`

```python


SparkContext.setCheckpointDir()


RDD.checkpoint()


RDD.isCheckpointed()


RDD.getCheckpointFile()
```

In [ ]:
sc.setCheckpointDir('losrudos/')

In [ ]:
doc = sc.textFile('all-shakespeare.txt')

In [ ]:
words = doc.flatMap( lambda x: x.split()) \
    .map( lambda x: (x, 1))\
    .reduceByKey( lambda x, y: x + y) 


In [ ]:
words.checkpoint()

In [ ]:
words.count()

In [ ]:
words.isCheckpointed()

In [ ]:
words.getCheckpointFile()

In [ ]:
sc.stop()

-----

Ejercicio: Checkpointing